# Main

In [1]:
from ColorMap import *
from tqdm import tqdm 
import pandas as pd
import cv2
from PSO import *

[[1, 0], [2, 3]]


In [6]:
N = 20
G = 50
sum_g = 0 #合計世代数
omega = 0.8
c1 = 2.6
c2 = 1.0
D = 2
up_lim = 5.0
lo_lim = -5.0
func = griewank_function
pso = PSO(N, omega, c1, c2, D, up_lim, lo_lim, func)
pso.init_phase()

In [7]:
for g in tqdm(range(G)):
    pso.solution_search_phase()

100%|██████████| 50/50 [00:00<00:00, 2282.69it/s]
